# [TUTORIAL] Constrained Task Planning (Sweep)

This tutorial is written to instruct basic usage of the task & motion planning pipeline  
One Indy7 robot and several environment geometries will be added and floor-wiping task will be conducted.  
Here, the wiping task is defined as 1) contact with floor, 2) starting waypoint, 3) goal waypoint.  
Thus, any motion that satisfies the constraint will be generated; it may not look like real wiping.  

## set running directory to project source

In [ ]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

## init combined robot config

In [ ]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, ((0,0,0), (0,0,0)),
                INDY_IP)]
              , connection_list=[False])

## create scene builder

In [ ]:
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None)
# # deprecated: s_builder.reset_reference_coord(ref_name="floor")
gscene = s_builder.create_gscene(crob)

## init planning pipeline

In [ ]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

```
open web ui on <your ip>:8050
click geometry items / Handles / Binders to highlight geometry on RVIZ
other functions may be buggy.. please report
```

## add environment

In [ ]:
from pkg.geometry.geometry import *

floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (0.3,0.5,0.01), (0.5,0,-0.005), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=False)
floor_col = gscene.create_safe(GEOTYPE.BOX, "floor_col", "base_link", (0.3,0.5,0.01), (0.5,0,-0.006), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=False, fixed=True, collision=True)
wall = gscene.create_safe(GEOTYPE.BOX, "wall", "base_link", (3,3,0.01), (-0.2,0,0), 
                           rpy=(0,np.pi/2,), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
goal = gscene.create_safe(GEOTYPE.BOX, "goal", "base_link", (0.1,0.1,0.01), (0.3,-0.4,-0.005), 
                          rpy=(0,0,0), color=(0.8,0.8,0.2,1), display=True, fixed=True, collision=False)
goal_col = gscene.create_safe(GEOTYPE.BOX, "goal_col", "base_link", (0.1,0.1,0.01), (0.3,-0.4,-0.006), 
                          rpy=(0,0,0), color=(0.8,0.8,0.2,1), display=False, fixed=True, collision=True)

In [ ]:
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True, exclude_link=["panda1_link7"])

In [ ]:
from pkg.utils.code_scraps import add_indy_gripper_asm2
add_indy_gripper_asm2(gscene, "indy0")

## add wp

In [ ]:
wp11 = gscene.create_safe(GEOTYPE.BOX, "wp11", "base_link", (0.08,0.08,0.01), (0.6,0.2,-0.005),rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)
wp12 = gscene.create_safe(GEOTYPE.BOX, "wp12", "base_link", (0.08,0.08,0.01), (0.6,-0.2,-0.005), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)
line1 = gscene.create_safe(GEOTYPE.BOX, "wline1", "base_link", (0.01,0.5,1e-6), (0.6,0,0), rpy=(0,np.pi/2,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)

wp21 = gscene.create_safe(GEOTYPE.BOX, "wp21", "base_link", (0.08,0.08,0.01), (0.5,0.2,-0.005),rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)
wp22 = gscene.create_safe(GEOTYPE.BOX, "wp22", "base_link", (0.08,0.08,0.01), (0.5,-0.2,-0.005), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)
line2 = gscene.create_safe(GEOTYPE.BOX, "wline2", "base_link", (0.01,0.5,1e-6), (0.5,0,0), rpy=(0,np.pi/2,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)

wp31 = gscene.create_safe(GEOTYPE.BOX, "wp31", "base_link", (0.08,0.08,0.01), (0.4,0.2,-0.005),rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)
wp32 = gscene.create_safe(GEOTYPE.BOX, "wp32", "base_link", (0.08,0.08,0.01), (0.4,-0.2,-0.005), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)
line3 = gscene.create_safe(GEOTYPE.BOX, "wline3", "base_link", (0.01,0.5,1e-6), (0.4,0,0), rpy=(0,np.pi/2,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)

## add brush

In [ ]:
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_body", link_name="base_link", dims=(0.1,0.07,0.02), 
                   center=(0.3,-0.4,0.04), rpy=(0,0,0), color=(0.7,0.7,0.3,1), display=True, collision=True, fixed=False)
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_handle", link_name="base_link", dims=(0.1,0.03,0.05), center=(0,0,0.035), rpy=(0,0,0), 
                   color=(0.7,0.7,0.3,1), display=True, collision=True, fixed=False, parent="brush_body")
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_face_col", link_name="base_link", dims=(0.1,0.06,0.028), center=(0,0,-0.025), rpy=(0,0,0), 
                   color=(0.8,0.8,0.8,1), display=True, collision=False, fixed=False, parent="brush_body")
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_face", link_name="base_link", dims=(0.1,0.06,0.03), center=(0,0,-0.025), rpy=(0,0,0), 
                   color=(0.8,0.8,0.8,1), display=True, collision=False, fixed=False, parent="brush_body")

## add box

In [ ]:
gbox1 = gscene.create_safe(gtype=GEOTYPE.BOX, name="box1", link_name="base_link", dims=(0.05,0.05,0.05), 
                   center=(0.5,0.1,0.025), rpy=(0,0,0), color=(0.7,0.3,0.3,1), display=True, collision=True, fixed=False)

gbox2 = gscene.create_safe(gtype=GEOTYPE.BOX, name="box2", link_name="base_link", dims=(0.05,0.05,0.05), 
                   center=(0.4,-0.1,0.025), rpy=(0,0,0), color=(0.7,0.3,0.3,1), display=True, collision=True, fixed=False)

## Register binders

In [ ]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepTool

In [ ]:
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name="indy0_tcp", 
                 dims=(0.01,)*3, center=(0,0,0.14), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

In [ ]:
pscene.create_binder(bname="grip0", gname="grip0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="floor", gname="floor", _type=PlacePlane)
pscene.create_binder(bname="goal_bd", gname="goal", _type=PlacePlane, point=(0,0,0.005), rpy=(0,0,0))
pscene.create_binder(bname="brush_face", gname="brush_face", _type=SweepTool, point=(0,0,-0.015), rpy=(0,0,0))

## add objects

In [ ]:
from pkg.planning.constraint.constraint_subject import CustomObject, Grasp2Point, PlacePoint, SweepPoint, SweepTask, BoxObject

In [ ]:
brush_handle = gscene.NAME_DICT["brush_handle"]
brush_face = gscene.NAME_DICT["brush_face"]
brush = pscene.create_subject(oname="brush", gname="brush_body", _type=CustomObject, 
                             action_points_dict = {"handle": Grasp2Point("handle", brush_handle, [0,0,0], [np.pi/2,0,0]),
                                                   "face": PlacePoint("face", brush_face, [0,0,-0.015], [0,0,0])})

In [ ]:
box1 = pscene.create_subject(oname="box1", gname="box1", _type=BoxObject, hexahedral=True)
box2 = pscene.create_subject(oname="box2", gname="box2", _type=BoxObject, hexahedral=True)

In [ ]:
from pkg.planning.constraint.constraint_common import MotionConstraint
from pkg.planning.constraint.constraint_subject import AbstractTask
from pkg.planning.constraint.constraint_subject import SweepLineTask

In [ ]:
sweep1 = pscene.create_subject(oname="sweep1", gname="floor", _type=SweepLineTask, 
                             action_points_dict = {"wp11": SweepPoint("wp11", wp11, [0,0,0.005], [0,0,0]),
                                                   "wp12": SweepPoint("wp12", wp12, [0,0,0.005], [0,0,0])},
                            geometry_vertical = line1)
sweep2 = pscene.create_subject(oname="sweep2", gname="floor", _type=SweepLineTask, 
                             action_points_dict = {"wp21": SweepPoint("wp21", wp21, [0,0,0.005], [0,0,0]),
                                                   "wp22": SweepPoint("wp22", wp22, [0,0,0.005], [0,0,0])},
                            geometry_vertical = line2)
sweep3 = pscene.create_subject(oname="sweep3", gname="floor", _type=SweepLineTask, 
                             action_points_dict = {"wp31": SweepPoint("wp31", wp31, [0,0,0.005], [0,0,0]),
                                                   "wp32": SweepPoint("wp32", wp32, [0,0,0.005], [0,0,0])},
                            geometry_vertical = line3)

### planners

In [ ]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)
mplan.update_gscene()
from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()
ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

## motion filters

In [ ]:
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
from pkg.planning.filtering.latticized_filter import LatticedChecker
TOOL_LINK = "indy0_tcp"
TOOL_LINK_BUNDLE = ["indy0_tcp", "indy0_link6"]

gcheck = GraspChecker(pscene)
rcheck = ReachChecker(pscene)
checkers_all = [gcheck, rcheck]
# lcheck = LatticedChecker(pscene, gcheck)
# checkers_all.append(lcheck)

In [ ]:
mplan.motion_filters = checkers_all

## Test plan

In [ ]:
from pkg.planning.constraint.constraint_common \
            import sample_redundancy, combine_redundancy
gtimer = GlobalTimer.instance()
initial_state = pscene.initialize_state(crob.home_pose)

# Node Sampler

In [ ]:
from pkg.planning.sampling.node_sampling import UniformNodeSampler

In [ ]:
tplan.new_node_sampler = UniformNodeSampler(0.5)
tplan.parent_node_sampler = UniformNodeSampler(0.5)

In [ ]:
gtimer.reset()
goal_nodes = [("floor", "floor", "goal", 2, 2, 2)]
gtimer.tic("plan")
ppline.search(initial_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, timeout_loop=300, multiprocess=True, timeout=1, timeout_constrained=1)
gtimer.toc("plan")
schedules = ppline.tplan.find_schedules()
schedules_sorted = ppline.tplan.sort_schedule(schedules)
snode_schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])

In [ ]:
print(gtimer)

## play searched plan

In [ ]:
ppline.play_schedule(snode_schedule[:2], period=0.01)
pscene.set_object_state(initial_state)
gscene.show_pose(crob.home_pose)
time.sleep(0.5)
gscene.show_pose(crob.home_pose)

In [ ]:
ppline.play_schedule(snode_schedule, period=0.03)

In [ ]:
len(snode_schedule)

In [ ]:
for i_s,  snode in enumerate(snode_schedule):
    print("{}: {}".format(i_s, snode.state.node))

## NOTE
* 위에 가운데 열 어떻게  0,1,1,1,1,2가 나오지? -> 일방통행 작업 플래그 추가
* 균등 샘플링 - 샘플 할때마다 노드별/전환별 확률 조정
* goal-directed extension 추가.

In [ ]:
tplan.node_dict[('grip0', 'goal', 0, 1, 2)]

## extend preserving goal-matching items

In [ ]:
print(gtimer)

## extend only no reservation

In [ ]:
print(gtimer)

## no extension

In [ ]:
print(gtimer)

## extend_toward goal

In [ ]:
print(gtimer)